### 유튜브 크롤링 

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyautogui
import time
from bs4 import BeautifulSoup

browser = webdriver.Chrome()
keyword = pyautogui.prompt("검색어를 입력하세요>>")

url = f'https://www.youtube.com/results?search_query={keyword}'

browser.implicitly_wait(10)
browser.maximize_window()
browser.get(url)

# 7번 스크롤하기
scroll_count = 3
i = 1
while True:
    #맨 아래로 스크롤 내린다
    browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)   
    
    time.sleep(2)

    if i == scroll_count:
        break
    i += 1

## Selenium - BeautifulSoup 연동하기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
infos = soup.select("div.text-wrapper") # 박스하나 

for info in infos:
    # 원하는 정보를 가져오기 
    # 제목
    title = info.select_one("a#video-title").text

    try:
        # 조회수
        #span class 2개로 구분이안될때 -> 상위 타입을 찾아 그 하위 타입의 1번째 2번째로 구분해서 가져온다
        views = info.select_one("div#metadata-line >  span:nth-child(3)").text

        #날짜
        date = info.select_one("div#metadata-line >  span:nth-child(4)").text
    except:
        views = "조회수 0회"
        date = "날짜 없음"

    print(title, views, date)
    

### 엑셀 저장하기

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyautogui
import time
from bs4 import BeautifulSoup
import openpyxl

browser = webdriver.Chrome()
keyword = pyautogui.prompt("검색어를 입력하세요>>")

# 엑셀 생성
wb = openpyxl.Workbook()
ws = wb.create_sheet(keyword)
ws.append(['번호', '제목', '조회수', '날짜'])

ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 30
ws.column_dimensions['D'].width = 30

url = f'https://www.youtube.com/results?search_query={keyword}'

browser.implicitly_wait(10)
browser.maximize_window()
browser.get(url)

# 7번 스크롤하기
scroll_count = 3
i = 1
while True:
    #맨 아래로 스크롤 내린다
    browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)   
    
    time.sleep(2)

    if i == scroll_count:
        break
    i += 1

## Selenium - BeautifulSoup 연동하기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
infos = soup.select("div.text-wrapper") # 박스하나 

for index, info in enumerate(infos,1):
    # 원하는 정보를 가져오기 
    # 제목
    title = info.select_one("a#video-title").text

    try:
        # 조회수
        #span class 2개로 구분이안될때 -> 상위 타입을 찾아 그 하위 타입의 1번째 2번째로 구분해서 가져온다
        views = info.select_one("div#metadata-line >  span:nth-child(3)").text

        #날짜
        date = info.select_one("div#metadata-line >  span:nth-child(4)").text
    except:
        views = "조회수 0회"
        date = "날짜 없음"

    print(title, views, date)
    ws.append([index, title, views,date])


wb.save(f'{keyword} 크롤링.xlsx')

    

### 데이터 전처리하기 

In [4]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyautogui
import time
from bs4 import BeautifulSoup
import openpyxl

def text_to_num(text):
    text = text.replace('조회수', '').replace('회','').strip() 
    if "만" in text:
        num = float(text.replace("만", "")) * 10000

    elif "억" in text:
        num = float(text.replace("억", "")) * 100000000   

    elif "천" in text:
        num = float(text.replace("천", "")) * 1000

    elif "없음" == text:
        num = 0

    else:
        num = int(text)

    return int(num)

    

browser = webdriver.Chrome()
keyword = pyautogui.prompt("검색어를 입력하세요>>")

# 엑셀 생성
wb = openpyxl.Workbook()
ws = wb.create_sheet(keyword)
ws.append(['번호', '제목', '조회수', '날짜'])

ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 30
ws.column_dimensions['D'].width = 30

url = f'https://www.youtube.com/results?search_query={keyword}'

browser.implicitly_wait(10)
browser.maximize_window()
browser.get(url)

# 7번 스크롤하기
scroll_count = 3
i = 1
while True:
    #맨 아래로 스크롤 내린다
    browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)   
    
    time.sleep(2)

    if i == scroll_count:
        break
    i += 1

## Selenium - BeautifulSoup 연동하기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
infos = soup.select("div.text-wrapper") # 박스하나 

for index, info in enumerate(infos,1):
    # 원하는 정보를 가져오기 
    # 제목
    title = info.select_one("a#video-title").text
    
    try:
        # 조회수
        #span class 2개로 구분이안될때 -> 상위 타입을 찾아 그 하위 타입의 1번째 2번째로 구분해서 가져온다(핵심)
        views = info.select_one("div#metadata-line >  span:nth-child(3)").text

        #날짜
        date = info.select_one("div#metadata-line >  span:nth-child(4)").text
    except:
        views = "조회수 0회"
        date = "날짜 없음"

    views = text_to_num(views)

    print(title, views, date)
    ws.append([index, title, views,date])


wb.save(f'{keyword} 크롤링.xlsx')

    

The chromedriver version (115.0.5790.102) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (115.0.5790.170); currently, chromedriver 115.0.5790.170 is recommended for chrome 115.*, so it is advised to delete the driver in PATH and retry




코딩 존잘, 코딩 잘하는 사람 특징 5가지 - 코딩 잘하는 법, 이렇게만 하세요.
 43000 1년 전


💻코딩 쥰내 잘하는 개발자 특 ㅋㅋㅋㅋ #Shorts
 3060000 4개월 전


코딩 무조건 빨리 배우는 법 - 제발 프로그래밍 기초 부터 차례대로 시작하지 마세요
 440000 4년 전


무조건 공감되는 코딩 입문자 특징 ㅋㅋㅋ #Shorts
 2210000 10개월 전


프로그래밍 언어 공부법 혼자서 제대로 공부하는 가장 쉽고 간단한 방법
 8500 9개월 전


코딩 잘하는법 (개발자답게 코딩하려면?)
 150000 2년 전


코딩 처음 시작 방법? 독학 순서? 자세히 알려드립니다! l 현직 개발자가 말하는 Codidic
 150000 2년 전


50대에 돈한푼 안 들이고, 영어를 잘할 수밖에 없게 된 '3가지' 비결 (김미경 대표 2부)
 750000 3개월 전


코딩 교육 30년차가 알려주는 진짜 개발자 되는 방법 (ft. 이민석 교수님)
 17000 4일 전


천재 수학자들이 만들어 낸 미로를 탈출하는 방법들(우수법,좌수법 아님)
 1420000 9개월 전


코딩, 소프트웨어 시대 - 어렵지 않은 이야기, 코딩이란?
 280000 8년 전


프로그래밍 독학할때 자주 하는 실수 5가지
 540000 3년 전


프로그래밍 실력을 향상시키는 방법
 7800 2년 전


개발자가 실력을 향상시킬 방법은 ○○뿐이에요_우아한형제들 김민태의 이야기
 200000 4년 전


해커가 말하는 프로그래밍 공부하는 방법! | 진짜 몸에 베는 프로그래밍 공부법
 80000 3년 전


초등학생도 이해하는 프로그래밍 기초
 200000 2년 전


효율적 개발 공부 방법
 7000 1년 전


코딩을 한달만에 배울 수 있다고? - 독서실은 진짜 아니야 (ft. 현직 개발자)
 210000 4년 전


그냥 개발자 vs 쌉고수 개발자 차이 ㅋㅋㅋㅋㅋㅋ #shorts
 460000 4개월 전


코딩 입문자를 위한 시작할때 무조건 알아야할 상식 [